In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## Importing the libraries
import numpy as np
import pandas as pd
import os
try:
    import theano
except:
    !pip install Theano
import theano
import keras
import tensorflow
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
print(os.listdir("../input"))


In [ ]:
# Loading the dataset
data = pd.read_csv("../input/churn-modelling/Churn_Modelling.csv")

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
## 2 categorical Variables ... Gender and geography
data['Geography'].value_counts()

In [ ]:
data['Gender'].value_counts()

In [ ]:
#### we will be using Artificial Neural networks for doing Churn Modelling. ANN will give more weights to the Independent
## variable that have more impact on Dependent variable.
## creating metrics of features X and dependent varible y
X = data.iloc[:,3:13].values
y = data.iloc[:,13].values

In [ ]:
X

In [ ]:
y

In [ ]:
#### Encoding categorical Variables
#### Country and Gender Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])
#onehotencoder = OneHotEncoder(categorical_features = [1])

ct = ColumnTransformer([("Geography", OneHotEncoder(), [1])], remainder = 'passthrough')

X = ct.fit_transform(X)
X = X[:, 1:]


#sklearn library removed categorical_features parameter for onehotencoder class.

In [ ]:
X

In [ ]:
y

In [ ]:
### splitting the Dataset in Train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

In [ ]:
#### we need to do standard scaling as Deep Learning involves lots of computations to ease computations
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#### Part 2- Making ANN
### Import keras library and important packages
### we need to import 2 modules...sequential module to initialize our Neural Network and Dense module to built layers of ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Initializing the ANN
classifier = Sequential()

In [ ]:
# Adding the Input layer and First Hidden layer
#here no of nodes in input layer is 11 which is the no of Independent variables and no of nodes in the
## output layer is 1(dependent variable)..if the output variable is binary there is only 1 node in the output layer and so the
### average is 11+1/2 = 6 i.e; 6 nodes will be used....
#### init --> corresponds to the step 1 of stochastic gradient descent we need to randomly initialize weights as small numbers
## close to 0(zero) and we can randomly initialize with a uniform function...which will initialize the weights according to
## uniform ditribution with weights close to 0(zero)

classifier.add(Dense(output_dim = 6, init = 'uniform',activation = 'relu',input_dim = 11))

In [ ]:
#### Adding the second Hidden Layer becuase Deep Learning is defined as an Artificial Neural Network with many Hidden layers 
## no need to use input_dim in the second hidden layer after creating 1st Hidden layer
## relu for rectifier activation function
classifier.add(Dense(output_dim = 6, init = 'uniform',activation = 'relu'))

In [ ]:
### Adding the Output layer
### here output_dim = 1 node becoz our output variable is categorical variable with 1 Node
### sigmoid actiavtion is used in the output layer as we need to get probabilities of the customers leaving the bank
### If we have more than 2 categories in the dependent variable then the activation function used in the output layer is softmax
### softmax is a sigmoid activation function for dependent variable with more than 2 categories
## and Output_dim = 3 for 3 categories in the dependent variable
classifier.add(Dense(output_dim = 1, init = 'uniform',activation = 'sigmoid'))

In [ ]:
# compiling the ANN
classifier.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
## Now we are going to choose a number of epochs that is the no of times we are training our ANN on the whole training set
## we are going to see this stochastic gradient descent in action that is we are going to see how our ANN model is trained 
## and how it is improving its accuracy at each round that is at each epoch
### Fitting the ANN to the training set
classifier.fit(X_train,y_train,batch_size = 10, nb_epoch = 100)

In [ ]:
## Predicting the test set results
y_pred = classifier.predict(X_test)

In [ ]:
y_pred

In [ ]:
## Now we need to convert the predicted probabilities in the form true or false 0 or 1 in order to use confusion Matrix
## we need to use a Threshold for this..we predict 0 below threshold and 1 above the threshold...here we choose threshold of 0.5
y_pred = (y_pred > 0.5)

In [ ]:
### Making the confusion Matrics
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

In [ ]:
p = sns.heatmap(pd.DataFrame(cm), annot=True, cmap="YlGnBu" ,fmt='g')
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
accuracy = (1520+199)/(1520+199+206+75)
print(accuracy)

In [ ]:
### We got an accuracy of 86% on the Test data.....

from sklearn.metrics import roc_curve
y_pred1 = classifier.predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_pred1)
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr,tpr, label='ANN')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('ROC curve')
plt.show()

In [ ]:
#Area under ROC curve
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred1)

In [ ]:
### The area Under the curve is 86% which means the Model is good.....